## **Stochastic Simulation Algorithm (SSA)**

**Stochastic Kinetics of mRNA Molecules in a General Transcription Model**

*Yuntao Lu and Yunxin Zhang*

School of Mathematical Sciences, Fudan University, Shanghai 200433, China

Email: `yuntaolu22@m.fudan.edu.cn` and `xyz@fudan.edu.cn`

This script is written to time SSA, generating date used to plot Figure 4. 

In [1]:
import gillespy2
import numpy as np
import time

In [2]:
import Parameters_for_Figures

In [3]:
# ===================================================================
# Specify parameters in the model here.
# Input:
# D0 (np.ndarray): Transition rate matrix for non-transcription events
# D1 (np.ndarray): Transition rate matrix for transcription events
# ===================================================================

In [4]:
# D0=Parameters_for_Figures.D0_1
# D1=Parameters_for_Figures.D1_1

In [5]:
# D0=Parameters_for_Figures.D0_3a
# D1=Parameters_for_Figures.D1_3a

In [6]:
# D0=Parameters_for_Figures.D0_3b
# D1=Parameters_for_Figures.D1_3b

In [7]:
d=1

In [8]:
def SSA_time(D0,D1):
    
    N=D0.shape[0]
    
    class Transcription(gillespy2.Model):
        def __init__(self, parameter_values=None):
            # Initialize the model
            gillespy2.Model.__init__(self, name="Transcription")
    
            # N=D0.shape[0]
            
            # Parameters established
            params = []
            for i in range(N):
                for j in range(N):
                    if str(float(D1[i, j]))==0:
                        pass
                    param_name = f'p{i+1}_{j+1}'
                    param_value = str(float(D1[i, j]))
                    params.append(gillespy2.Parameter(name=param_name, expression=param_value))
            for i in range(N):
                for j in range(N):
                    if i == j:
                        continue
                    if str(float(D0[i, j]))==0:
                        pass
                    param_name = f't{i+1}_{j+1}'
                    param_value = str(float(D0[i, j])) 
                    params.append(gillespy2.Parameter(name=param_name, expression=param_value))
            params.append(gillespy2.Parameter(name='d', expression=1))
            self.add_parameter(params)
    
            # Species established
            mRNA = gillespy2.Species(name='mRNA', initial_value=0)
            state1 = gillespy2.Species(name='state1', initial_value=1)
            state=[]
            for i in range(2,N+1):
                state_name=f"state{i}"
                state.append(gillespy2.Species(name=state_name, initial_value=0))
            state.append(state1)
            state.append(mRNA)
            self.add_species(state)
            state_only=[]
            for i in range(1,N+1):
                state_only.append(f"state{i}")
    
            # Reaction Objects Built
            reactions = []
            for i in range(N):
                for j in range(N):
                    rxn_name = f"Pr{i+1}_{j+1}"
                    rate_param = f'p{i+1}_{j+1}'
                    reactions.append(gillespy2.Reaction(
                        name=rxn_name,
                        reactants={state_only[i]: 1},
                        products={state_only[j]: 1, mRNA: 1},
                        rate=rate_param
                    ))
            for i in range(N):
                for j in range(N):
                    if i==j:
                        continue
                    rxn_name = f"Tr{i+1}_{j+1}"
                    rate_param = f't{i+1}_{j+1}'
                    reactions.append(gillespy2.Reaction(
                        name=rxn_name,
                        reactants={state_only[i]: 1},
                        products={state_only[j]: 1},
                        rate=rate_param
                    ))
            reactions.append(gillespy2.Reaction(
                name="dec",
                reactants={mRNA: 1},
                products={},
                rate='d'
            ))
            self.add_reaction(reactions)
            
            # TimeSpan specifies the time points at which to keep data from a simulation
            self.timespan(np.linspace(0, 10, 2))
    # This is original SSA using Python solver NumPySSASolver from gillespy2.
    model= gillespy2.NumPySSASolver(model=Transcription())
    
    # This is Tau-Leaping simulation algorithm, an accelerated version of original SSA.
    # We use the Python solver TauLeapingSolver from gillespy2.
    # model= gillespy2.TauLeapingSolver(model=MAP())
    
    # Start timing the programme
    start_time = time.time()
    
    trajectories=1000
    
    model.run(number_of_trajectories=trajectories)
    
    # Timing ends here
    end_time = time.time()
    timing = end_time - start_time
    
    print(f"Running time for SSA (model of oder {N}; 1000 trajectories; simulation time T=10): {timing:.8f} seconds.")
    
    return timing

In [9]:
times=[]
for i in range(1,11):
    time1=SSA_time(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                     )
    time2=SSA_time(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                     )
    time3=SSA_time(Parameters_for_Figures.generate_D0n(i),
                     Parameters_for_Figures.generate_D1n(i),
                     )
    times.append(float(np.average([time1,time2,time3])))

Running time for SSA (model of oder 1; 1000 trajectories; simulation time T=10): 0.19171405 seconds.
Running time for SSA (model of oder 1; 1000 trajectories; simulation time T=10): 0.24007273 seconds.
Running time for SSA (model of oder 1; 1000 trajectories; simulation time T=10): 0.17619228 seconds.
Running time for SSA (model of oder 2; 1000 trajectories; simulation time T=10): 0.63108492 seconds.
Running time for SSA (model of oder 2; 1000 trajectories; simulation time T=10): 0.61968350 seconds.
Running time for SSA (model of oder 2; 1000 trajectories; simulation time T=10): 0.61241889 seconds.
Running time for SSA (model of oder 3; 1000 trajectories; simulation time T=10): 1.64292359 seconds.
Running time for SSA (model of oder 3; 1000 trajectories; simulation time T=10): 1.43971467 seconds.
Running time for SSA (model of oder 3; 1000 trajectories; simulation time T=10): 0.90992785 seconds.
Running time for SSA (model of oder 4; 1000 trajectories; simulation time T=10): 3.52258992

In [10]:
# Save the results in a `.npy` file
# np.save('SSA_times.npy', times)

In [11]:
# load .npy file
loaded_array = np.load('SSA_times.npy')
loaded_list_from_array = loaded_array.tolist()
print(loaded_list_from_array)

[0.20265968640645346, 0.621062437693278, 1.330855369567871, 2.9570671717325845, 4.772596120834351, 7.023046890894572, 11.802446365356445, 18.32266648610433, 28.08046881357829, 39.72367787361145]
